In [1]:
import sys
sys.path.insert(0, '..')
from modules import constants
from modules.transform import rescale_target
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import pickle
import os

In [8]:
X_train = pd.read_feather(os.path.join(constants.prepared_data_path, 'X_train.arrow'))
y_train = pd.read_feather(os.path.join(constants.prepared_data_path, 'y_train.arrow'))
X_val = pd.read_feather(os.path.join(constants.prepared_data_path, 'X_val.arrow'))
y_val = pd.read_feather(os.path.join(constants.prepared_data_path, 'y_val.arrow'))
X_test = pd.read_feather(os.path.join(constants.prepared_data_path, 'X_test.arrow'))

In [9]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,162
0,0.872511,1.290274,1.458728,0.629730,-0.030403,-0.595110,-0.561763,-0.878354,-0.758569,-0.838021,...,-1.306688,-1.197441,-1.303965,-1.282607,-1.194492,-1.015856,-1.311786,-1.238816,-1.226393,-1.169501
1,-1.193387,-0.534692,-0.631937,-0.892989,1.434542,0.838514,-1.300640,0.514047,-0.259633,-0.197469,...,-0.625552,-0.184951,0.612140,1.144531,1.235965,1.010498,0.748056,0.343691,0.196776,-0.241122
2,1.058913,0.582676,1.304842,1.091153,-1.263007,-0.851297,0.745481,-0.745745,0.239303,-0.325579,...,0.886624,0.746863,0.532150,0.258280,-0.018515,-0.281787,-0.461440,-0.367618,0.099405,0.461111
3,-1.524486,-0.472661,-0.719057,-1.082891,0.523806,0.286283,-0.902783,0.580352,-0.883303,-0.325579,...,1.319339,1.516274,1.841413,1.876061,1.723720,1.709636,1.841799,1.972430,1.825209,1.530155
4,0.480528,0.787051,-0.199426,-0.969458,-0.645514,-0.798509,-1.073293,-1.408793,-1.319872,-1.286408,...,1.250627,1.092834,0.946688,0.818143,0.749755,0.704004,0.661390,0.783108,1.070465,1.258077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44386,0.880978,1.212131,0.825117,-0.181356,-0.069842,-0.559468,0.006604,0.182523,0.114569,0.058752,...,-0.738751,-0.711379,-0.562908,-0.547515,-0.615256,-0.713751,-0.615277,-0.653513,-0.653292,-0.658670
44387,-1.067931,4.250450,2.819889,0.005315,-0.510230,-0.945737,-1.698497,-0.480525,-1.943542,-1.094242,...,1.244682,1.204063,1.139123,1.063923,1.078069,1.048713,1.168179,1.354674,1.441403,1.465478
44388,1.623788,-0.025209,0.853023,1.647940,-1.043366,-0.651098,1.257011,0.646657,0.925340,0.763360,...,-0.798033,-0.708640,-0.429717,0.119381,0.463217,0.895149,1.257074,1.603889,0.819123,-0.313464
44389,0.236466,-0.796018,-0.888326,-1.340631,0.238620,0.493116,-0.220743,0.381438,0.488771,0.507139,...,-1.210425,-1.090596,-0.769107,-0.792310,-0.553739,-0.089102,0.174305,0.077615,-0.643808,-1.135793


In [10]:
y_train

,0,1,2,3,4,5
0,0.258657,-0.008937,-0.006761,-0.013854,-0.009254,-0.010495
1,-0.957565,-0.007895,-0.006761,-0.014037,-0.008758,-0.012653
2,1.337716,-0.009369,-0.006753,-0.008138,-0.008646,-0.007245
3,-0.024598,-0.009987,-0.006749,-0.013939,-0.007039,-0.013850
4,-0.545991,-0.009244,-0.006761,-0.014042,-0.008997,-0.013183
...,...,...,...,...,...,...
44386,0.546600,-0.009392,-0.006758,-0.013493,-0.008815,-0.012994
44387,-0.980155,-0.008780,-0.006762,-0.014042,-0.008780,-0.013799
44388,0.139293,-0.008656,-0.006761,-0.013810,-0.009023,-0.007837
44389,-0.132420,-0.009172,-0.006762,-0.014039,-0.008626,-0.014033


In [10]:
lm = LinearRegression()
lm.fit(X_train, y_train)
print(r2_score(y_val, lm.predict(X_val)))
print(r2_score(rescale_target(y_val), rescale_target(lm.predict(X_val))))



0.02285955573956953
0.02285955573956962


array([[ 5.33869705e-01,  4.68667010e+00,  9.32206505e+04,
        -5.93048676e+03,  6.38451134e+01,  3.63947407e+03],
       [ 4.57137965e-01,  3.38168426e+02, -7.00974917e+04,
         2.56858161e+03, -9.68947301e+01, -1.07369305e+04],
       [ 4.77965550e-01,  7.91966820e+02, -1.05396758e+05,
        -1.08763709e+04, -7.27118884e+01,  5.91109083e+04],
       ...,
       [ 4.39575611e-01, -1.22997411e+02,  8.67118826e+03,
        -7.60324134e+03,  9.43980166e+01,  9.33037350e+03],
       [ 4.32876598e-01, -1.27759852e+02, -8.94151119e+03,
         2.06848160e+03, -1.82639137e+01,  3.31968636e+03],
       [ 5.72639246e-01,  5.13638659e+02,  1.09907164e+05,
         2.62741993e+04, -3.95826819e+00, -2.99295568e+04]])

In [30]:
test_preds = rescale_target(lm.predict(X_test))
test_df = pd.DataFrame(test_preds, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
test_df.insert(0, 'id', pd.read_csv(constants.test_metadata_path).id)
test_df.to_csv('../submission.csv', index=False)

In [33]:
 pd.read_csv(constants.train_metadata_path)

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X18_mean,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,0.117484,1.243779,1.849375,50.216034,0.008921,1.601473,0.025441,0.153608,0.279610,15.045054
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,0.389315,0.642940,1.353468,574.098472,0.003102,0.258078,0.000866,0.034630,0.010165,11.004477
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,8.552908,0.395241,2.343153,1130.096731,NaN,NaN,NaN,NaN,NaN,NaN
3,195728812,18.680834,1473.933350,163.100006,45.009758,381.053986,20.436666,120,131,125,...,1.083629,0.154200,1.155308,1042.686546,0.011692,2.818356,0.110673,0.011334,0.229224,141.857187
4,195251545,0.673204,530.088867,50.857777,38.230709,1323.526855,45.891998,91,146,120,...,0.657585,10.919966,2.246226,2386.467180,0.006157,1.128000,0.026996,0.553815,0.107092,87.146899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55484,190558785,19.472172,244.795914,39.127552,67.074493,472.710358,27.758673,118,140,131,...,0.233690,1.783193,1.608341,969.547831,NaN,NaN,NaN,NaN,NaN,NaN
55485,194523231,13.724150,1450.000000,162.260208,43.139324,652.716858,26.694387,125,144,135,...,1.017099,12.713048,2.418300,1630.015480,0.005474,0.128133,0.117010,3.164520,0.082212,136.503697
55486,195888987,14.741204,581.866638,109.231110,89.272148,507.273010,26.874668,118,155,136,...,2.717395,10.206478,2.722599,602.229880,0.019727,0.215040,0.156309,0.919139,0.079395,26.159626
55487,135487319,16.094763,1180.838135,80.176193,22.909716,342.184021,17.346189,109,130,117,...,4.429659,9.372170,3.251739,244.387170,NaN,NaN,NaN,NaN,NaN,NaN
